In [232]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

In [233]:
#baseline, this model gives F1 of 0.515
SVM_clfr = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clfr', SVC(kernel='linear', class_weight='balanced'))
    ])

#baseline w default class_weight, this model gives F1 of 0.37
# SVM_clfr = Pipeline([
#     ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clfr', SVC(kernel='linear'))
#     ])

#baseline w class_weight of .8/.2, this model gives F1 of 0
# SVM_clfr = Pipeline([
#     ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clfr', SVC(kernel='linear', class_weight={0:.8, 1:.2}))
#     ])

#baseline w class_weight of .2/.8, this model gives F1 of 0.502
# SVM_clfr = Pipeline([
#     ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clfr', SVC(kernel='linear', class_weight={0:.2, 1:.8}))
#     ])

In [234]:
#import training & test data, these are files Danny added to GH repo
common_train = pd.read_csv('train-split.tsv', sep = '\t')
common_test = pd.read_csv('test-split.tsv', sep = '\t')

In [235]:
#import list of drugs manually identified in train/test Tweets
drug_list = pd.read_csv('Bryans_drug_list.csv')

In [250]:
#funct to iterate through tweet_text & set boolean to True if drug name present, can also return the drug name
def flag_drug(tweet_text):
    tweet_text_lower = tweet_text.lower()
    split_tweet_text = tweet_text_lower.split()
    
#     drug_flag = False
    drug_flag = 0
    drug = False
    for i in split_tweet_text:
        if i in drug_list['Lower'].values:
#             drug_flag = True
            drug_flag = 1
            drug = i
            return drug_flag
    return drug_flag

In [251]:
#add col to train & test to flag Tweets with drugs
common_train['drug_flag'] = common_train['tweet_text'].apply(flag_drug)
common_test['drug_flag'] = common_test['tweet_text'].apply(flag_drug)

In [277]:
#fit model to training data
SVM_clfr = SVM_clfr.fit(common_train['tweet_text'],common_train['adr_mention'])
# SVM_clfr = SVM_clfr.fit(common_train[['tweet_text','tweet_id']],common_train['adr_mention']) #this throws a shape/# of samples error

In [278]:
#predicting against training data until we have the train/test split available
predicted = SVM_clfr.predict(common_test['tweet_text'])

In [279]:
common_test['predicted_label'] = predicted

In [280]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
f1 = f1_score(common_test['adr_mention'],predicted)
precision = precision_score(common_test['adr_mention'],predicted)
recall = recall_score(common_test['adr_mention'],predicted)
accuracy = accuracy_score(common_test['adr_mention'],predicted)

In [281]:
print('F1: {}\nPrecision: {}\nRecall: {}\nAccuracy: {}'.format(round(f1,3), round(precision,3), round(recall,3), round(accuracy,3)))

F1: 0.515
Precision: 0.431
Recall: 0.639
Accuracy: 0.886
